In [1]:
import torch 
torch.cuda.is_available()

False

In [2]:
%pip install datasets transformers huggingface_hub
!apt-get install git-lfs


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 51.2 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 MB 46.2 MB/s  0:00:006m0:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 11/12 [datasets]ess]  WARNING: The script datasets-cli is installed in '/usr/local/python/3.12.1/bin' which is not on PATH.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12/12 [datasets]
Note: you may need to restart the kernel to use updated packages.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


In [13]:
import pandas 
data = 'Restaurant_Reviews.tsv'
csv = pandas.read_table(data, sep ='\t')
print(csv)
csv.to_csv('Restaurant_Reviews.csv', index=False)


                                                Review  Liked
0                             Wow... Loved this place.      1
1                                   Crust is not good.      0
2            Not tasty and the texture was just nasty.      0
3    Stopped by during the late May bank holiday of...      1
4    The selection on the menu was great and so wer...      1
..                                                 ...    ...
995  I think food should have flavor and texture an...      0
996                           Appetite instantly gone.      0
997  Overall I was not impressed and would not go b...      0
998  The whole experience was underwhelming, and I ...      0
999  Then, as if I hadn't wasted enough of my life ...      0

[1000 rows x 2 columns]


In [14]:
from datasets import load_dataset
dataset = load_dataset('csv', data_files = 'Restaurant_Reviews.csv')
print(dataset)

Generating train split: 1000 examples [00:00, 43798.34 examples/s]

DatasetDict({
    train: Dataset({
        features: ['Review', 'Liked'],
        num_rows: 1000
    })
})


In [15]:
# train segment 
train_segment = dataset["train"].shuffle(seed=42).select([i for i in range(750)])

# test segment 
test_segment = dataset["train"].shuffle(seed=42).select([i for i in range(750, 825)])

print(train_segment)

print(test_segment)

Dataset({
    features: ['Review', 'Liked'],
    num_rows: 750
})
Dataset({
    features: ['Review', 'Liked'],
    num_rows: 75
})


In [16]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [17]:
def preprocess_function(examples): 
    return tokenizer(examples["Review"], truncation = True)

tokenized_train = train_segment.map(preprocess_function, batched = True)
tokenized_test = test_segment.map(preprocess_function, batched = True)

Map: 100%|██████████| 75/75 [00:00<00:00, 9899.07 examples/s]


In [18]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [19]:
from transformers import AutoModelForSequenceClassification
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
import numpy as np
%pip install evaluate

import evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
def compute_metrics(eval_pred): 
    load_accuracy = evaluate.load('accuracy')
    load_f1 = evaluate.load('f1')

    logits, labels = eval_pred 
    predictions = np.argmax(logits, axis = -1)
    accuracy = load_accuracy.compute(predictions=predictions, references=labels)['accuracy']
    f1 = load_f1.compute(predictions=predictions, references=labels)['f1']

    return (f'accuracy: ${accuracy}\n f1: ${f1}')


In [24]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
!pip install transformers[torch]
!pip install 'accelerate>=0.26.0'
from transformers import TrainingArguments, Trainer



In [ ]:
repo_name = "finetuning-sentiment-model-3000-samples"
 
training_args = TrainingArguments(
   output_dir=repo_name,
   learning_rate=2e-5,
   per_device_train_batch_size=16,
   per_device_eval_batch_size=16,
   num_train_epochs=2,
   weight_decay=0.01,
   save_strategy="epoch",
   push_to_hub=True,
)

trainer = Trainer(
   model=model,
   args=training_args,
   train_dataset=tokenized_train,
   eval_dataset=tokenized_test,
   tokenizer=tokenizer,
   data_collator=data_collator,
   compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.push_to_hub()

In [ ]:
bleeker_st_reviews = ['''
We went on Saturday night and we queued twice for around 5-10’. Not too bad.

The best pizza was the champions 🍄‍🟫 one although a bit too much cheese.

The classic pizza was good but the sauce a bit too sweet for us.

The pepperoni square pizza was not fully cooked. The dough didn’t feel fully cooked. We should have returned it as we didn’t finish it/enjoy it.

We would have preferred if instead of a ricotta, there would have been a pizza with a blue cheese like Gorgonzola for example.

The staff was super kind and the atmosphere inside/outside is very friendly.

Prices aren’t too expensive and there’s the option of buying slices, which we loved.

Best part? How thin and crunchy the dough was.''', 

'''I came here to try the Nonna Maria pizza and im so glad i did because it was very delicious! The ingredients were fresh, sauce was flavorful but not overpowering, and the crust was thin with just the right amount of crisp. Not greasy and it’s cooked perfectly. One of the best tasting pizza i ever had.

The space is tidy with many indoor and outdoor sitting. The staff were friendly and service was quick.

I’ll definitely be coming back!''', 
'''
On a corner of Bleecker Street you can visit euphoria if you so choose to. Bleecker Street Pizza is a flavorful punch of a slice, well positioned amongst the hierarchy of New York pizzerias. The inside is unassuming, a lot like a million other places but whatever is going on in that oven is the truth. When we stopped by there was a decent sized line but they do keep it pushing well. Despite the amount of people indoors the place is still pretty clean and there is an order to all the madness going on around you. Lining the walls they have photos of celebrities who made the trek (or swiped a Metrocard) to enjoy a slice here. Like any pizzeria they offer a lot of different slices but since we’ve heard this place was one of the best slices in the city we opted for a regular cheese slice, no frills to make sure we were hearing correctly.

After getting our slice and sitting outside (yes, they do have outdoor seating but it is hard to come by), we were ready to dig in. Here’s our careful analysis: the crust is undoubtedly New York style. Firm, crisp, not too much give and not burnt to high heavens like a New Haven slice. A little flop to it but a firm underbelly to the slice, which seems like an over evaluation, but trust that it is not. The first bite was incredible. The cheese is stretchy and light, the sauce rich but complimenting the cheese. Easily one of the 3 best pizza slices in our lives based on the first bite alone and by the final bite we were questioning if it was the best period. Needless to say, Bleecker Street Pizza is not an if you’re in the city or an if you’re in the area excursion; this is a go out of your way and eat this slice place. Phenomenal and trying to find excuses to come back for seconds ASAP.
''', 
'''
Just ok pizza nothing special overpriced cheese slice $4.80 . Little to no sauce a there are 6 guys behind the counter and the place is still dirty . They don’t keep inside clean . I ask the guy to give me a specific slice an he act like he switched it to give me the same slice he originally picked out. Shady
''', 
'''
First off. Best garlic knots around not gonna lie. One star though for the customer service. The way my family was treated ruined the whole experience. We walk in and immediately get yelled at to order. (Never been here before) and I’m not kidding we were not given even 15 seconds to look at anything before literally being yelled at. Actually yelled at too not just loudly spoken too. “WHAT DO YOU WANT? IF YOU AREN’T READY STEP ASIDE!” however there was nobody behind us and it was just three of us, they could have said hello, welcome in, order when you’re ready, get any slice you want, here are our options, etc. And then while one employee was yelling at us to get our order, the women on the register started asking us what we were getting as well. Never been in an establishment where two people took my order at the same time, so aggressively at that. The woman was rude as hell. Pizza isn’t bad at all, it’s just not worth the hype. The garlic knots are though but I will never return regardless.

''', 

'''
Best pizza I’ve had so far in NYC. If you’re in Manhattan, you need to stop in and get the Nona slice. It’s amazing. The crust is light and airy, and the toppings are so incredibly good!

''', 
'''
Bleecker Street Pizza is a NY classic. Visiting town and stopped in. The guys that run this shop are friendly but they offer few modifications so you either like what’s in the case or head to Joe’s down the street with more options. Personally, I think Bleecker is much better and I don’t think you can go wrong with any of the slices in the case but everyone has a picky friend. There’s plenty of room to stand at the table in the middle or the small side tables inside and out. You might wait a few minutes but pizza is a quick meal so table turnover is fast and the guys are good about wiping them down in between. They also sell white claw and beer for anyone looking for a drink with their slice.
''']

In [ ]:
from transformers import pipeline
 
sentiment_model = pipeline(model="federicopascual/finetuning-sentiment-model-3000-samples")
sentiment_model(bleeker_st_reviews)


